In [21]:
import re
import pandas as pd
from collections import Counter
from tqdm import tqdm

In [27]:
# trainfilename = '../../data/train.tsv'
# validfilename = '../../data/valid.tsv'
# testfilename = '../../data/personalized_test.tsv'
docsfilename = './news.tsv'
testfilename = './personalized_test.tsv'
# stop_words = set(stopwords.words('english'))

In [35]:
# WORD_FREQ_THRESHOLD = 3
# MAX_CONTENT_LEN = 500
# MAX_BODY_LEN = 100
# MAX_TITLE_LEN = 16
# WORD_EMBEDDING_DIM = 300
MAX_CLICK_LEN = 50

# word2freq = {}
# word2index = {}

In [19]:
def word_tokenize(sent):
    pat = re.compile(r'[\w]+|[.,!?;|]')
    if isinstance(sent, str):
        return pat.findall(sent.lower())
    else:
        return []

In [7]:
#edit
news_id_hl = {}

def read_news(filename,filer_num=3):
    news={}
    
    category, subcategory=[], []
    news_index={}
    index=1
    word_cnt=Counter()
    err = 0
    news_data = pd.read_csv(filename, sep='\t')
    news_data.fillna(value=" ", inplace=True)
    for i in tqdm(range(len(news_data))):
        doc_id,vert,_, title, snipplet= news_data.loc[i,:][:5]
        news_index[doc_id]=index
        index+=1
        
        #edit
        news_id_hl[doc_id] = title
        
        title = title.lower()
        title = word_tokenize(title)
        snipplet = snipplet.lower()
        snipplet = word_tokenize(snipplet)
        category.append(vert)
        news[doc_id] = [vert,title,snipplet]     
        word_cnt.update(snipplet+title)
    # 0: pad; 1: <sos>; 2: <eos>
    word = [k for k , v in word_cnt.items() if v >= filer_num]
    word_dict = {k:v for k, v in zip(word, range(3,len(word)+3))}
    category=list(set(category))
    category_dict={k:v for k, v in zip(category, range(1,len(category)+1))}

    return news,news_index,category_dict,word_dict

In [33]:
def Doc2ID(doclist,news2id):
    return [news2id[i] for i in doclist if i in news2id ]

In [30]:
def PadDoc(doclist):
    if len(doclist) >= MAX_CLICK_LEN:
        return doclist[-MAX_CLICK_LEN:]
    else:
        return [0] * (MAX_CLICK_LEN-len(doclist)) + doclist[:MAX_CLICK_LEN]

In [22]:
%time news,news_index,category_dict,word_dict = read_news(docsfilename)

100%|████████████████████████████████████████████████████████████████████████████████████████| 113762/113762 [00:57<00:00, 1971.76it/s]


CPU times: user 1min 24s, sys: 11.4 s, total: 1min 35s
Wall time: 1min 37s


In [31]:
def parse_test_user(filename,news_index):
        
    df = pd.read_csv(filename, sep='\t')
    
    df['clicknewsID'] = df['clicknewsID'].apply(lambda x: PadDoc(Doc2ID(x.split(','),news_index)))
    
    df['posnewID']  = df['posnewID'].apply(lambda x: Doc2ID(x.split(','),news_index))
    
    df['rewrite_titles'] = df['rewrite_titles'].apply(lambda x: [i.lower() for i in x.split(';;')] )
    
    pos_lists = []
    for userindex, (pos_lis, rewrite_title_lis) in enumerate(zip(df['posnewID'].values.tolist(), df['rewrite_titles'].values.tolist())):
        for pos, rewrite_title in zip(pos_lis, rewrite_title_lis):
            if rewrite_title.strip() == '':
                continue
            else:
                pos_lists.append([userindex, pos, rewrite_title])
    
    return df['clicknewsID'].values.tolist(), pos_lists

In [36]:
%time TestUsers, TestSamples = parse_test_user(testfilename,news_index)

CPU times: user 1.97 s, sys: 524 ms, total: 2.49 s
Wall time: 2.63 s


Addition of model generated summaries to the dataset

In [43]:
# opening the file in read mode
my_file = open("hyps_NRMS.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
model_summ = data.split("\n")
my_file.close()

In [44]:
setup_dict = []

key_list = list(news_index.keys())
# print(key_list)
val_list = list(news_index.values())

# print key with val 100
position = val_list.index(100)

#model summary index
idx = 0
#model_id = 1 for pens nrms
model_id = 1

for sample in TestSamples:
    dic = {}
    uid = "NT"+str(sample[0]+1)
    doc_idx = sample[1]
#     news_id_idx = val_list.index(doc_idx)
#     print(news_id_idx)
    position = val_list.index(sample[1])
    news_id_idx = key_list[position]
#     print(news_id_idx)
    grf_summ = sample[2]
#     original_hl = news[news_id_idx]
    
    dic = {'user_id': uid, 'doc_id': news_id_idx, 'user_summ': grf_summ, 'doc_summ': news_id_hl[news_id_idx], 
           'model_id': model_id, 'model_'+str(model_id)+'_summ': model_summ[idx]}
    idx = idx + 1
    
    setup_dict.append(dic)

In [45]:
# opening the file in read mode
my_file = open("hyps_NAML.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
model_summ = data.split("\n")
# print(model_summ)
my_file.close()

In [46]:
# setup_dict = []

key_list = list(news_index.keys())
# print(key_list)
val_list = list(news_index.values())

# print key with val 100
position = val_list.index(100)

#model summary index
idx = 0
#model_id = 1 for pens naml
model_id = 2

for sample in TestSamples:
    dic = {}
    uid = "NT"+str(sample[0]+1)
    doc_idx = sample[1]
#     news_id_idx = val_list.index(doc_idx)
#     print(news_id_idx)
    position = val_list.index(sample[1])
    news_id_idx = key_list[position]
#     print(news_id_idx)
    grf_summ = sample[2]
#     original_hl = news[news_id_idx]
    
    dic = {'user_id': uid, 'doc_id': news_id_idx, 'user_summ': grf_summ, 'doc_summ': news_id_hl[news_id_idx], 
           'model_id': model_id, 'model_'+str(model_id)+'_summ': model_summ[idx]}
    idx = idx + 1
    
    setup_dict.append(dic)

In [47]:
# opening the file in read mode
my_file = open("hyps_EBNR.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
model_summ = data.split("\n")
# print(model_summ)
my_file.close()

In [48]:
# setup_dict = []

key_list = list(news_index.keys())
# print(key_list)
val_list = list(news_index.values())

# print key with val 100
position = val_list.index(100)

#model summary index
idx = 0
#model_id = 3 for pens ebnr
model_id = 3

for sample in TestSamples:
    dic = {}
    uid = "NT"+str(sample[0]+1)
    doc_idx = sample[1]
#     news_id_idx = val_list.index(doc_idx)
#     print(news_id_idx)
    position = val_list.index(sample[1])
    news_id_idx = key_list[position]
#     print(news_id_idx)
    grf_summ = sample[2]
#     original_hl = news[news_id_idx]
    
    dic = {'user_id': uid, 'doc_id': news_id_idx, 'user_summ': grf_summ, 'doc_summ': news_id_hl[news_id_idx], 
           'model_id': model_id, 'model_'+str(model_id)+'_summ': model_summ[idx]}
    idx = idx + 1
    
    setup_dict.append(dic)

In [49]:
# opening the file in read mode
my_file = open("hyps_NRMS_2.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
model_summ = data.split("\n")
# print(model_summ)
my_file.close()

In [50]:
# setup_dict = []

key_list = list(news_index.keys())
# print(key_list)
val_list = list(news_index.values())

# print key with val 100
position = val_list.index(100)

#model summary index
idx = 0
#model_id = 3 for pens ebnr
model_id = 4

for sample in TestSamples:
    dic = {}
    uid = "NT"+str(sample[0]+1)
    doc_idx = sample[1]
#     news_id_idx = val_list.index(doc_idx)
#     print(news_id_idx)
    position = val_list.index(sample[1])
    news_id_idx = key_list[position]
#     print(news_id_idx)
    grf_summ = sample[2]
#     original_hl = news[news_id_idx]
    
    dic = {'user_id': uid, 'doc_id': news_id_idx, 'user_summ': grf_summ, 'doc_summ': news_id_hl[news_id_idx], 
           'model_id': model_id, 'model_'+str(model_id)+'_summ': model_summ[idx]}
    idx = idx + 1
    
    setup_dict.append(dic)

In [51]:
# opening the file in read mode
my_file = open("hyps_NAML_2.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
model_summ = data.split("\n")
# print(model_summ)
my_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'hyps_NAML_2.txt'

In [ ]:
# setup_dict = []

key_list = list(news_index.keys())
# print(key_list)
val_list = list(news_index.values())

# print key with val 100
position = val_list.index(100)

#model summary index
idx = 0
#model_id = 3 for pens ebnr
model_id = 5

for sample in TestSamples:
    dic = {}
    uid = "NT"+str(sample[0]+1)
    doc_idx = sample[1]
#     news_id_idx = val_list.index(doc_idx)
#     print(news_id_idx)
    position = val_list.index(sample[1])
    news_id_idx = key_list[position]
#     print(news_id_idx)
    grf_summ = sample[2]
#     original_hl = news[news_id_idx]
    
    dic = {'user_id': uid, 'doc_id': news_id_idx, 'user_summ': grf_summ, 'doc_summ': news_id_hl[news_id_idx], 
           'model_id': model_id, 'model_'+str(model_id)+'_summ': model_summ[idx]}
    idx = idx + 1
    
    setup_dict.append(dic)

In [ ]:
# opening the file in read mode
my_file = open("hyps_EBNR_2.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
model_summ = data.split("\n")
# print(model_summ)
my_file.close()

In [ ]:
# setup_dict = []

key_list = list(news_index.keys())
# print(key_list)
val_list = list(news_index.values())

# print key with val 100
position = val_list.index(100)

#model summary index
idx = 0
#model_id = 3 for pens ebnr
model_id = 6

for sample in TestSamples:
    dic = {}
    uid = "NT"+str(sample[0]+1)
    doc_idx = sample[1]
#     news_id_idx = val_list.index(doc_idx)
#     print(news_id_idx)
    position = val_list.index(sample[1])
    news_id_idx = key_list[position]
#     print(news_id_idx)
    grf_summ = sample[2]
#     original_hl = news[news_id_idx]
    
    dic = {'user_id': uid, 'doc_id': news_id_idx, 'user_summ': grf_summ, 'doc_summ': news_id_hl[news_id_idx], 
           'model_id': model_id, 'model_'+str(model_id)+'_summ': model_summ[idx]}
    idx = idx + 1
    
    setup_dict.append(dic)

In [ ]:
setup_dict[0:5]

In [ ]:
setup_dict[-1]

In [ ]:
datalist = setup_dict

In [ ]:
dataset = {}

for sample in datalist:
    doc_id = sample['doc_id']
    if sample['model_id'] == 1:
        if doc_id in dataset.keys() and ('1' in dataset[doc_id]['m_dict'].keys()):    
            dataset[doc_id]['u_dict'].update({sample['user_id']: sample['user_summ']})
            dataset[doc_id]['m_dict']['1'].update({sample['user_id']: sample['model_1_summ']})
            dataset[doc_id]['m_dict']['2'].update({sample['user_id']: ' '})
            dataset[doc_id]['m_dict']['3'].update({sample['user_id']: ' '})
            dataset[doc_id]['m_dict']['4'].update({sample['user_id']: ' '})
#             dataset[doc_id]['m_dict']['5'].update({sample['user_id']: ' '})
            dataset[doc_id]['m_dict']['6'].update({sample['user_id']: ' '})
        else:
            dic = {}
            u_dict, m_dict = {}, {}
            u_dict[sample['user_id']] = sample['user_summ']
            model_dict = {}
            m_dict[sample['user_id']] = sample['model_1_summ']

            model_dict.update({'1': m_dict, '2':{}, '3': {}, '4':{}, '6': {}})
            dic = {'doc_text': news[doc_id], 'doc_summ': sample['doc_summ'], 'u_dict': u_dict, 'm_dict': model_dict}    
            dataset[doc_id] = dic
    
    elif sample['model_id'] == 2:
            
            dataset[doc_id]['m_dict']['2'].update({sample['user_id']: sample['model_2_summ']})

    elif sample['model_id'] == 3 :
            dataset[doc_id]['m_dict']['3'].update({sample['user_id']: sample['model_3_summ']})
    
    elif sample['model_id'] == 4 :
            dataset[doc_id]['m_dict']['4'].update({sample['user_id']: sample['model_4_summ']})
    
#     elif sample['model_id'] == 5 :
#             dataset[doc_id]['m_dict']['5'].update({sample['user_id']: sample['model_5_summ']})
    
    elif sample['model_id'] == 6 :
            dataset[doc_id]['m_dict']['6'].update({sample['user_id']: sample['model_6_summ']})